In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Аналіз A/B тесту

Маємо проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це класична гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку. У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми проаналізуємо вплив на утримання (retention) гравців. Тобто хочемо зрозуміти чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Змінні в даних наступні:

- userid - унікальний номер, який ідентифікує кожного гравця.
- version - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- sum_gamerounds - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- retention_1 - чи через 1 день після встановлення гравець повернувся і почав грати?
- retention_7 - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [3]:
import numpy as np
import pandas as pd


In [4]:
df = pd.read_csv('drive/MyDrive/data_csv/cookie_cats.csv')

In [12]:
df

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [14]:
retention7_means = df.groupby('version')['retention_7'].mean()

In [15]:
retention7_means

,retention_7
version,
gate_30,0.190201
gate_40,0.182000


***Згідно отриманих середніх значень показників утримання гравців для двох груп можна припустити, що контрольна група gate_30 має краще утримання гравців на протязі 7 днів.***

2. Перевірте з допомогою z-тесту аналогічно до прикладу в лекції, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для двох вибірок. Виведіть результат у форматі:
```
z statistic: ...
p-value: ...
Довірчий інтервал 95% для групи control: [..., ...]
Довірчий інтервал 95% для групи treatment: [..., ...]
```
де замість `...` - обчислені значення. В якості висновка дайте відповідь на два питання:  
    1. чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
    2. чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
    
Зверніть увагу, в такому і схожому завданнях ми використовуєм `proportion` Z-тест. Це тому що в нас залежна змінна має бінарне значення (повернеться аби ні користувач, чи клікне або ні користувач в інших ситуаціях - всього два можливих значення в змінної: 0/1, True/False ). Якщо б ми вимірювали скажімо чи є стат. значущою різниця між вагою чоловіків і жінок в певній вибірці, ми б використовувавли функцію `statsmodels.stats.ztest`, бо залежна змінна `вага` є неперервною (тип float, замість типу int чи bool і тільки двох можливих значень).

In [16]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [24]:
#вибираємо користувачів для контрольної і дослідної групи
control_gr = df[df['version'] == 'gate_30']
treatment_gr = df[df['version'] == 'gate_40']

In [26]:
#рахуємо скільки в сумі вийшло утриманих користувачів по групах
control_success = df[df['version'] == 'gate_30']['retention_7'].sum()
treatment_success = df[df['version'] == 'gate_40']['retention_7'].sum()

In [25]:
#рахуємо загальну кількість користувачів
n_con = control_gr.count()
n_treat = control_gr.count()

In [34]:
#записуємо необхідні дані для тесту у змінні
count = [control_success, treatment_success]
nobs = [n_con[1],n_treat[1]]

<ipython-input-34-94c1ef4efeac>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  nobs = [n_con[1],n_treat[1]]


In [37]:
z_stat, pval = proportions_ztest(count, nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(count, nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'Довірчий інтервал 95% для групи control: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'Довірчий інтервал 95% для групи treatment: [{lower_treat:.3f}, {upper_treat:.3f}]')


z statistic: 1.91
p-value: 0.056
Довірчий інтервал 95% для групи control: [0.187, 0.194]
Довірчий інтервал 95% для групи treatment: [0.182, 0.189]


**1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?**
*Значення p є близьким до статистично значущого 0,05, але не є ним, тож згідно даного дослідження формально різниця не є статистично значущою, але відхилити 0 гіпотезу з високою впевненістю ми не можемо.*

**2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?**
*Довірчі інтервали для двох різних груп частково перетинаються, це каже про те, що є різниця в утриманні, але незначна, щоб впевнено сказати що якась з версій краще.*

3. Є ще один тип тестів, який використовується для бінарної метрики як от "зробить юзер дію, чи ні" - тест [**Хі-квадрат**](https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/8-chi-squared-tests) (ще одне [пояснення](https://www.scribbr.com/statistics/chi-square-tests/) тесту з прикладами). В нього інші гіпотези Н0 і Н1 на відміну від z- та t-тестів. А також цей тест можна використовувати, якщо в нас більше за 2 досліджувані групи, тобто в нас не А/В тест, а А/B/C/D, наприклад.  

В **z- та t-тестах** (які відрізняються тим, що ми в першому не знаємо дисперсію генеральної сукупності, але якщо в нас великий набір даних, то ці два тести дають дуже схожі результати) **ми перевіряємо, чи є різниця у середніх показниках по групам користувачів**.  

А в **тесті Хі-квадрат ми перевіряємо чи є звʼязок між групою користувача і тим, чи він зробить цікаву нам дію**. Це ніби дослідження одного і того самого, але дещо різними способами. Для перевірки, можна виконувати кілька тестів (особливо, якщо один дає якийсь непереконливий результат типу р-значення 0.07 - наче і fail to regect H0 на рівні стат значущості 5%, але цікаво, що скажуть інші тести), тож, зробимо і ми тест хі-квадрат та порівняємо його результат з z-тестом.

Про різницю між тестами можна почитати ще [тут](https://stats.stackexchange.com/a/178860) - це просто пояснення користувача стековерфлоу, але там розумні люди сидять.

Для проведення хі-квадрат тесту скористаємось функцією з `scipy.stats` `chi2_contingency` для обчислення статистики хі-квадрат і р-значення для перевірки конкретної гіпотези. У цю функцію вам треба передати таблицю 2х2: кількість випадків для кожної версії гри і значення `retention_7`.

**Задача**: виконайте тест хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та тим, чи зайде гравець на 7ий день після встановлення гри.
Тут гіпотези наступні
- Н0: значення retention_7 не залежить від версії гри
- Н1: є залежність між версією гри і значенням retention_7

Виведіть p-значення та зробіть висновок.


In [1]:
from scipy.stats import chi2_contingency

In [18]:
data_for_chi2 = pd.crosstab(df['version'], df['retention_7'])

In [21]:
data_for_chi2

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [22]:
chi2_stat, p_value, dof, expected = chi2_contingency(data_for_chi2)

In [25]:
print(f"Статистика хі-квадрат: {chi2_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Очікувані частоти:\n{expected}")

Статистика хі-квадрат: 9.9591
p-value: 0.0016
Очікувані частоти:
[[36382.90257127  8317.09742873]
 [37025.09742873  8463.90257127]]


***На основі отриманого p-value = 0.0016, яке менше за статистичну значущість 5% і відповідно ми можемо зробити висновок, що є статистично значуща залежність між розміщенням воріт і утриманням гравців. Отжі нульова гіпотеза може бути відхилена в цьому випадку.***